In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import KFold

C:\Users\jiguo\AppData\Local\Enthought\Canopy\User\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
        self.S_train=[]
        self.S_test=[]
        self_folds=[]
        
    def fit(self, X_train, y_train):
        X_train=np.array(X_train)
        y_train=np.array(y_train)
        self.folds = list(KFold(len(y_train), n_folds=self.n_folds, shuffle=True, random_state=17))
        self.S_train = np.zeros((X_train.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            for j, (train_idx, test_idx) in enumerate(self.folds):
                X_CVtrain = X_train[train_idx]
                y_CVtrain = y_train[train_idx]
                X_CVholdout = X_train[test_idx]
                # y_holdout = y[test_idx]
                clf[j].fit(X_CVtrain, y_CVtrain)
                y_pred = clf[j].predict(X_CVholdout)[:]
                self.S_train[test_idx, i] = y_pred
        self.stacker.fit(self.S_train, y_train)
        
    def predict(self, X_test):    
        X_test=np.array(X_test)
        self.S_test = np.zeros((X_test.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((X_test.shape[0], len(self.folds)))
            for j in range(len(self.folds)):
                S_test_i[:, j] = clf[j].predict(X_test)[:]
            self.S_test[:, i] = S_test_i.mean(1)
        y_pred = self.stacker.predict(self.S_test)[:]
        return y_pred

In [ ]:
n=1000
X=pd.DataFrame(np.random.randn(n,1))
y=X.iloc[:,0]+.2*pd.Series(np.random.randn(n))
X_train,y_train=X.iloc[:n/2], y.iloc[:n/2]
X_test, y_test=X.iloc[n/2:], y.iloc[n/2:]
from sklearn.linear_model import LinearRegression, Ridge
n_folds=5
ensembleObj=Ensemble(n_folds=n_folds, stacker=LinearRegression(fit_intercept=False, n_jobs=7), \
                     base_models=[[LinearRegression() for i in np.arange(n_folds)],\
                                  [Ridge() for i in np.arange(n_folds)]])
ensembleObj.fit(X_train=X_train, y_train=y_train)
ensembleObj.predict(X_test=X_test)